### LoRA Fine-tuning Gemma-2B

This notebook is made for LoRA fine-tuning Gemma-2B. LoRA is a parameter efficient fine-tuning technique that only adjusts few parameters instead of full fine-tuning of the model, thus, it's faster. We will be using [VMWare/open-instruct](https://huggingface.co/datasets/VMware/open-instruct) dataset that has instructions. To apply LoRA, we'll use [PEFT](https://huggingface.co/docs/peft/index) library and for supervised instruction tuning, we will use `SFTTrainer` from [TRL](https://huggingface.co/docs/trl/en/index).

In [28]:
!python3 -m venv .venv
!source .venv/bin/activate
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Mar 16 15:57:06 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 546.01       CUDA Version: 12.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  Off |
|  0%   42C    P8    22W / 450W |  10412MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
!pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
!pip install -q -U transformers peft accelerate datasets trl bitsandbytes scipy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "vc_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)


Login to Hugging Face Hub, since Gemma-2B has gated access and login confirms that you have access to the model. If you don't have an access, get it from the model repository [here](https://huggingface.co/google/gemma-2b) your request will shortly be accepted.

In [33]:
from huggingface_hub import login
import os
from dotenv import load_dotenv
load_dotenv()
login(token=os.getenv('HF_TOKEN'))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/katopz/.cache/huggingface/token
Login successful


We'll shrink the model even further by loading it in 4bit using `bitsandbytes`. Then initialize the model with the CausalLM head and initialize the tokenizer.

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os

model_id = "sail/Sailor-7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/15.4G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Load the dataset.

In [36]:
from datasets import load_dataset

data = load_dataset("json", data_files="./ava-dataset-gemma.json" , split="train")


In [37]:
data

Dataset({
    features: ['text_column'],
    num_rows: 55
})

Depending on your dataset prompts, you might want to truncate and handle overflowing tokens like below. If you keep it like this, your prompts will be truncated though and you will have bad results. 😔 So adjust the below cell depending on what you need.

In [38]:
def tokenize_dataset(ds):
  result = tokenizer(ds["text_column"],truncation=True,
                       max_length=512)
  #sample_map = result.pop("overflow_to_sample_mapping")
  #for key, values in ds.items():
  #  result[key] = [values[i] for i in sample_map]
  #  print(result[key])
  return result

In [39]:
ds = data.map(tokenize_dataset)

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [40]:
ds

Dataset({
    features: ['text_column', 'input_ids', 'attention_mask'],
    num_rows: 55
})

In [41]:
ds[1]

{'text_column': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{มูลค่าของตลาดคริปโตในไตรมาสที่ 2 ของปี 2023 เป็นยังไง}\n\n### Response:ในส่วนของ Shanghai Upgrade ในช่วงเดือนเมษายน โดยหลังจากเสร็จสิ้นการอัปเกรดจริง ได้เกิดการ Sell On Fact ประกอบกับปัญหาต่าง ๆ  ที่เกิดขึ้นระหว่าง Centralized Exchange (CEX) กับทาง SEC ในเรื่องของ Security Token ประกอบกับทางฝั่ง Binance ได้มีการลาออกของผู้บริหารระดับ C-Level พร้อมทั้งมีข่าวการปลดพนักงานออกจากทาง Binance มากกว่า 1,000 ตำแหน่ง ทำให้สร้างความกดดันให้กับตลาดอีกครั้ง โดยมูลค่าของตลาดคริปโตในไตรมาสที่ 2 ไปทำจุดสูงสุดที่ $1.236B ก่อนที่จะปรับตัวลดลงมา $994B (ลดลง 19.57%) ในวันที่ 14 มิถุนายน โดยในวันที่ 16 มิถุนายนตลาดได้รับข่าวดีอีกครั้งจากทาง Blackrock ที่ยื่นขอเปิด Bitcoin ETF กับทาง SEC ทำให้ Sentiment ตลาดกลับมาดีอีกครั้ง ทำให้ตลาดปิดไตรมาสที่ 2 ยังคงบวกเล็กน้อยที่ 0.78%',
 'input_ids': [38214,
  374,
  458,
  7600,
  429,
  16555,
  264,
  3383,
  13,
  9645,
 

Initializing `SFTTrainer` from TRL is all you need!

Small note: if your dataset needs formatting, you can write a formatting function and pass it. You need to either pass `formatting_func` or `dataset_text_field` if your dataset text field doesn't need any formatting and you did your preprocessing beforehand.

Then simply call ` train`. Note that this notebook is built for educational purposes so you might need to adjust the hyperparameters to your own use case.

In [42]:
import transformers
from trl import SFTTrainer


trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    dataset_text_field="text_column",
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    #formatting_func=formatting_func,
)
trainer.train()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/55 [00:00<?, ? examples/s]

Step,Training Loss
1,3.428300
2,2.973800
3,2.197900
4,1.984400
5,2.043900
6,1.498700
7,1.833800
8,1.674800
9,1.406000
10,1.352400


TrainOutput(global_step=100, training_loss=0.6907850787043571, metrics={'train_runtime': 93.2807, 'train_samples_per_second': 4.288, 'train_steps_per_second': 1.072, 'total_flos': 3094350870896640.0, 'train_loss': 0.6907850787043571, 'epoch': 7.27})

In [43]:
text = "ตัว Stablecoin ที่มีการเติบโตมากที่สุดในปี 2023 คืออะไร ### Response:"

device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


ตัว Stablecoin ที่มีการเติบโตมากที่สุดในปี 2023 คืออะไร ### Response:USDC ของ Circle นั้นถือว่าทำได้ดีตัวหนึ่ง โดยในปี 2023 นั้น Circle ได้มีการ List ตัว USDC บน Nasdaq Clearing Firm ซึ่งจะทำให้ Circle มีรายได้เพิ่มขึ้นจากค่า Fees ที่ Nasdaq จ่ายให้ในแต่ละ


เฉลย: Stablecoin ถือว่าเป็นตัวกลางที่ถูกใช้ในโลกคริปโตในการใช้แลกเปลี่ยนซื้อ-ขาย เหรียญอื่น ๆ  ที่มีอยู่ในตลาด โดยจากภาพรวม เราจะเห็นว่ามูลค่ารวมของ Stablecoin นั้นยังมีแนวโน้มลดลงอยู่ จากตอนต้นปีที่มีขนาดอยู่ที่ $137.77B ลดลงมาอยู่ที่ $124.12B (ลดลง -9.91%) อ้างอิงข้อมูล ณ วันที่ 1 พฤศจิกายน 2023